<a href="https://colab.research.google.com/github/lucaszsilva1/sistemas_distribuido_paralelo/blob/master/Sistemas_distribuido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Serial simples

In [ ]:
import time
import os
import random

# --- Funções de Utilidade ---

def criar_grade(linhas, colunas):
    """Cria uma grade 2D preenchida com 0s."""
    return [[0 for _ in range(colunas)] for _ in range(linhas)]

def popular_grade_aleatoriamente(grade):
    """Preenche a grade com estados vivos (1) ou mortos (0) aleatoriamente."""
    for i in range(len(grade)):
        for j in range(len(grade[0])):
            grade[i][j] = random.randint(0, 1)

def imprimir_grade(grade):
    """Imprime a grade no console de forma visual."""
    os.system('cls' if os.name == 'nt' else 'clear') # Limpa o console
    for linha in grade:
        for celula in linha:
            print('■' if celula == 1 else ' ', end=' ')
        print()
    print("-" * (len(grade[0]) * 2))

# --- Lógica Principal do Jogo da Vida ---

def contar_vizinhos_vivos(grade, x, y):
    """Conta os 8 vizinhos vivos de uma célula em (x, y) usando uma grade toroidal."""
    linhas = len(grade)
    colunas = len(grade[0])
    contador = 0
    # Itera sobre a vizinhança 3x3 ao redor da célula
    for i in range(-1, 2):
        for j in range(-1, 2):
            # Ignora a própria célula central
            if i == 0 and j == 0:
                continue

            # Calcula as coordenadas do vizinho com o "wrap-around" (toroidal)
            vizinho_x = (x + i + linhas) % linhas
            vizinho_y = (y + j + colunas) % colunas

            contador += grade[vizinho_x][vizinho_y]

    return contador

def calcular_proxima_geracao(grade_atual):
    """Calcula o estado da próxima geração com base na grade atual."""
    linhas = len(grade_atual)
    colunas = len(grade_atual[0])
    nova_grade = criar_grade(linhas, colunas)

    for i in range(linhas):
        for j in range(colunas):
            vizinhos_vivos = contar_vizinhos_vivos(grade_atual, i, j)
            estado_atual = grade_atual[i][j]

            # Aplicando as regras do Jogo da Vida
            if estado_atual == 1: # Célula viva
                if vizinhos_vivos < 2 or vizinhos_vivos > 3:
                    nova_grade[i][j] = 0 # Morre por solidão ou superpopulação
                else:
                    nova_grade[i][j] = 1 # Continua viva
            else: # Célula morta
                if vizinhos_vivos == 3:
                    nova_grade[i][j] = 1 # Nasce por reprodução

    return nova_grade

# --- Execução Principal ---

if __name__ == "__main__":
    LINHAS = 20
    COLUNAS = 40
    GERACOES = 100
    INTERVALO = 0.1 # segundos entre gerações

    grade = criar_grade(LINHAS, COLUNAS)
    popular_grade_aleatoriamente(grade)

    for geracao in range(GERACOES):
        print(f"Geração: {geracao + 1}/{GERACOES}")
        imprimir_grade(grade)
        grade = calcular_proxima_geracao(grade)
        time.sleep(INTERVALO)

# Paralelismo com trheads

In [ ]:
import threading
import time
import os
import random
import time

# Reutilizaremos as funções: criar_grade, popular_grade_aleatoriamente,
# imprimir_grade e contar_vizinhos_vivos do exemplo anterior.
# Para simplificar, elas estão copiadas aqui.

# --- Funções de Utilidade (mesmas da Parte 1) ---
def criar_grade(l, c): return [[0 for _ in range(c)] for _ in range(l)]
def popular_grade_aleatoriamente(g):
    for i in range(len(g)):
        for j in range(len(g[0])): g[i][j] = random.randint(0, 1)
def imprimir_grade(g):
    os.system('cls' if os.name == 'nt' else 'clear')
    for r in g:
        for c in r: print('■' if c == 1 else ' ', end=' ')
        print()
    print("-" * (len(g[0]) * 2))
def contar_vizinhos_vivos(g, x, y):
    l, c = len(g), len(g[0])
    count = 0
    for i in range(-1, 2):
        for j in range(-1, 2):
            if i == 0 and j == 0: continue
            nx, ny = (x + i + l) % l, (y + j + c) % c
            count += g[nx][ny]
    return count

# --- Lógica Paralela ---

def worker_calcular_slice(grade_atual, nova_grade, linha_inicio, linha_fim):
    """Função alvo da thread: calcula uma fatia horizontal da grade."""
    for i in range(linha_inicio, linha_fim):
        for j in range(len(grade_atual[0])):
            vizinhos_vivos = contar_vizinhos_vivos(grade_atual, i, j)
            estado_atual = grade_atual[i][j]

            if estado_atual == 1:
                if vizinhos_vivos < 2 or vizinhos_vivos > 3:
                    nova_grade[i][j] = 0
                else:
                    nova_grade[i][j] = 1
            else:
                if vizinhos_vivos == 3:
                    nova_grade[i][j] = 1

def calcular_proxima_geracao_paralela(grade_atual, num_threads):
    """Cria e gerencia as threads para calcular a próxima geração."""
    linhas = len(grade_atual)
    colunas = len(grade_atual[0])
    nova_grade = criar_grade(linhas, colunas)

    threads = []
    # Divide as linhas entre o número de threads
    slice_size = linhas // num_threads

    for i in range(num_threads):
        linha_inicio = i * slice_size
        # A última thread pega o resto das linhas
        linha_fim = (i + 1) * slice_size if i != num_threads - 1 else linhas

        # Cria a thread com a função worker e seus argumentos
        thread = threading.Thread(
            target=worker_calcular_slice,
            args=(grade_atual, nova_grade, linha_inicio, linha_fim)
        )
        threads.append(thread)
        thread.start() # Inicia a execução da thread

    # Sincronização: espera todas as threads terminarem
    for thread in threads:
        thread.join()

    return nova_grade

# --- Execução Principal ---

if __name__ == "__main__":
    LINHAS = 40
    COLUNAS = 80
    GERACOES = 200
    INTERVALO = 0.05
    NUM_THREADS = 4 # Experimente mudar este valor

    grade = criar_grade(LINHAS, COLUNAS)
    popular_grade_aleatoriamente(grade)

    start_time = time.perf_counter()

    for geracao in range(GERACOES):
        print(f"Geração: {geracao + 1}/{GERACOES} (usando {NUM_THREADS} threads)")
        imprimir_grade(grade)
        grade = calcular_proxima_geracao_paralela(grade, NUM_THREADS)
        time.sleep(INTERVALO)

    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"\nTempo total de execução: {execution_time:.4f} segundos")

# Sockets

In [ ]:
# Célula única para o Google Colab

import socket
import pickle
import time
import os
import random
import threading

# ==============================================================================
# 1. FUNÇÕES DE UTILIDADE E LÓGICA DO JOGO (Compartilhadas por todos)
# ==============================================================================

def criar_grade(linhas, colunas):
    """Cria uma grade 2D preenchida com 0s."""
    return [[0 for _ in range(colunas)] for _ in range(linhas)]

def popular_grade_aleatoriamente(grade):
    """Preenche a grade com estados vivos (1) ou mortos (0) aleatoriamente."""
    for i in range(len(grade)):
        for j in range(len(grade[0])):
            grade[i][j] = random.randint(0, 1)

def imprimir_grade(grade, geracao_info):
    """Imprime a grade no console de forma visual."""
    os.system('cls' if os.name == 'nt' else 'clear')
    print(geracao_info)
    print("+" + "---" * len(grade[0]) + "+")
    for linha in grade:
        print("|", end="")
        for celula in linha:
            print(' ■ ' if celula == 1 else '   ', end='')
        print("|")
    print("+" + "---" * len(grade[0]) + "+")

def contar_vizinhos_vivos(grade, x, y):
    """Conta os 8 vizinhos vivos de uma célula em (x, y) usando uma grade toroidal."""
    linhas, colunas = len(grade), len(grade[0])
    contador = 0
    for i in range(-1, 2):
        for j in range(-1, 2):
            if i == 0 and j == 0:
                continue
            vizinho_x = (x + i + linhas) % linhas
            vizinho_y = (y + j + colunas) % colunas
            contador += grade[vizinho_x][vizinho_y]
    return contador

def calcular_fatia(grade_completa, linha_inicio, linha_fim):
    """Calcula apenas as linhas designadas da grade."""
    colunas = len(grade_completa[0])
    fatia_resultado = criar_grade(linha_fim - linha_inicio, colunas)

    for i in range(linha_inicio, linha_fim):
        for j in range(colunas):
            vizinhos_vivos = contar_vizinhos_vivos(grade_completa, i, j)
            estado_atual = grade_completa[i][j]
            nova_linha_na_fatia = i - linha_inicio
            if estado_atual == 1:
                fatia_resultado[nova_linha_na_fatia][j] = 1 if 2 <= vizinhos_vivos <= 3 else 0
            else:
                fatia_resultado[nova_linha_na_fatia][j] = 1 if vizinhos_vivos == 3 else 0
    return fatia_resultado

# ==============================================================================
# 2. LÓGICA DO SERVIDOR E DO WORKER (Agora como funções para rodar em threads)
# ==============================================================================

def rodar_servidor(host, port, num_workers, geracoes, intervalo, finalizar_evento):
    """Função que encapsula toda a lógica do servidor."""
    worker_conns = []

    # Cria o socket do servidor
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((host, port))
        s.listen(num_workers)
        print(f"✅ Servidor escutando em {host}:{port}. Aguardando {num_workers} worker(s)...")

        # MODIFICAÇÃO: Em vez de um loop infinito com Ctrl+C, o servidor agora espera
        # exatamente o número de workers que planejamos iniciar.
        for _ in range(num_workers):
            conn, addr = s.accept()
            worker_conns.append(conn)
            print(f"👷 Worker conectado de {addr}")

        print("\n🚀 Todos os workers conectados. Iniciando simulação...")
        time.sleep(1) # Pequena pausa para tudo se estabilizar

        # Inicia a simulação
        linhas, colunas = 20, 30
        grade = criar_grade(linhas, colunas)
        popular_grade_aleatoriamente(grade)
        slice_size = linhas // num_workers

        for geracao in range(geracoes):
            info = f"Geração: {geracao + 1}/{geracoes} (Distribuído com {num_workers} workers)"
            imprimir_grade(grade, info)

            # 1. Enviar trabalho para cada worker
            for i, conn in enumerate(worker_conns):
                linha_inicio = i * slice_size
                linha_fim = (i + 1) * slice_size if i != num_workers - 1 else linhas
                dados = {"grade": grade, "inicio": linha_inicio, "fim": linha_fim}
                conn.sendall(pickle.dumps(dados))

            # 2. Coletar resultados
            nova_grade = criar_grade(linhas, colunas)
            for i, conn in enumerate(worker_conns):
                data_bytes = conn.recv(4096)
                fatia_calculada = pickle.loads(data_bytes)
                linha_inicio = i * slice_size
                linha_fim = (i + 1) * slice_size if i != num_workers - 1 else linhas
                nova_grade[linha_inicio:linha_fim] = fatia_calculada["fatia"]

            grade = nova_grade
            time.sleep(intervalo)

    # MODIFICAÇÃO: Sinaliza para os workers que a simulação terminou
    finalizar_evento.set()
    print("🏁 Simulação concluída. Servidor encerrando.")
    # Fecha as conexões para que as threads dos workers possam terminar
    for conn in worker_conns:
        conn.close()

def rodar_worker(host, port, finalizar_evento):
    """Função que encapsula a lógica do worker."""
    # MODIFICAÇÃO: Pequeno delay para garantir que o servidor já está escutando
    # antes de o worker tentar se conectar. Evita race conditions.
    time.sleep(0.5)

    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.connect((host, port))
            # MODIFICAÇÃO: O loop agora verifica o evento de finalização
            while not finalizar_evento.is_set():
                try:
                    # Recebe os dados em pedaços e junta
                    data_bytes = s.recv(4096)
                    if not data_bytes: # Conexão fechada pelo servidor
                        break

                    dados_recebidos = pickle.loads(data_bytes)

                    fatia_resultado = calcular_fatia(
                        dados_recebidos["grade"],
                        dados_recebidos["inicio"],
                        dados_recebidos["fim"]
                    )

                    dados_para_servidor = {"fatia": fatia_resultado}
                    s.sendall(pickle.dumps(dados_para_servidor))
                except (ConnectionResetError, BrokenPipeError):
                    # Ocorre quando o servidor fecha a conexão abruptamente no final
                    break
    except ConnectionRefusedError:
        print("❌ Worker não conseguiu se conectar ao servidor.")


# ==============================================================================
# 3. ORQUESTRAÇÃO PRINCIPAL (Inicia as threads do servidor e dos workers)
# ==============================================================================

if __name__ == '__main__':
    # --- Parâmetros da Simulação ---
    HOST = '127.0.0.1'
    PORT = 65432
    NUM_WORKERS = 4  # Você pode mudar este número
    GERACOES = 50
    INTERVALO = 0.2

    # MODIFICAÇÃO: Cria um evento para sinalizar o fim da simulação
    finalizar_evento = threading.Event()

    # Cria e inicia a thread do servidor
    servidor_thread = threading.Thread(
        target=rodar_servidor,
        args=(HOST, PORT, NUM_WORKERS, GERACOES, INTERVALO, finalizar_evento)
    )
    servidor_thread.start()

    # Cria e inicia as threads dos workers
    worker_threads = []
    for _ in range(NUM_WORKERS):
        worker_thread = threading.Thread(target=rodar_worker, args=(HOST, PORT, finalizar_evento))
        worker_threads.append(worker_thread)
        worker_thread.start()

    # Espera a thread do servidor terminar
    servidor_thread.join()
    # Espera todas as threads dos workers terminarem
    for wt in worker_threads:
        wt.join()

    print("\n✅ Todas as threads foram finalizadas. Programa encerrado.")